In [ ]:
!pip install tensorflow_addons

In [ ]:
# importing required libraries
import tensorflow as tf 
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2
from skimage import color
import scipy
import math

import sklearn
from sklearn.model_selection import train_test_split

import keras
from keras import callbacks
from keras import optimizers
#from keras.engine import Model
from keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/drive')

In [ ]:
def rescale(i, image, width, height):
  new_im = cv2.resize(image,(width,height))
  # normalize values to be between -1 and 1
  # new_im = (new_im - 127.5 )/127.5
  # normalize values to be between 0 and 1
  # new_im = new_im/255
  return new_im

In [ ]:
# We first load the necessary libraries, the dataset and reshape its dimensons

# FOR INITIAL RAW IMAGES
faulty_dir = '/content/drive/Shareddrives/Senior Thesis/Casting_image/casting_512x512/casting_512x512/def_front/'
ok_dir = '/content/drive/Shareddrives/Senior Thesis/Casting_image/casting_512x512/casting_512x512/ok_front/'

# FOR AUGMENTED IMAGES
#faulty_dir = '/content/drive/Shareddrives/Senior Thesis/Casting_image/casting_data/casting_data/images/def_front/'
#ok_dir = '/content/drive/Shareddrives/Senior Thesis/Casting_image/casting_data/casting_data/images/ok_front/'
directories = (faulty_dir, ok_dir)

# check the total number of images we have
count = 0
for direct in directories:
  for file in os.listdir(direct):
    count += 1

# set the values of the X and y arrays
num_images = count
dim = 280
input_shape = (dim, dim, 3)
height = input_shape[0]
width = input_shape[1]
channels = input_shape[2]

X = np.zeros((num_images,height,width,channels))
y = np.zeros((num_images))

# populate the arrays
i = 0
j = -1
for direct in directories:
  j+=1
  for file in os.listdir(direct):
    path = (direct+file)
    im = plt.imread(path)
    scaled_im = rescale(i, im, width, height)
    #edges, gradient = filter(scaled_im[:,:,0])
    X[i] = scaled_im
    #X[i,:,:,1] = edges/255
    #X[i,:,:,2] = (gradient-np.min(gradient))/(np.max(gradient)-np.min(gradient))
    if j == 0:
      # 1 for faulty
      y[i] = 1
    else:
      # 0 for ok image
      y[i] = 0
    i+=1

In [ ]:
convert = False
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering

if convert:
  dims = 50
  X_train = X
  y_train = y
  X_train_fit = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]*X_train.shape[3])
  pca = PCA(n_components = dims)
  X_train_fit = pca.fit_transform(X_train_fit)
  #kmeans clustering on PCA data
  data = AgglomerativeClustering(n_clusters = 3, compute_distances=True).fit(X_train_fit)

  idx_train0 = np.array(np.where(y_train==0)).flatten()
  idx_train1 = np.array(np.where(y_train==1)).flatten()
  idx_labels0 = np.array(np.where(data.labels_==0)).flatten()
  idx_labels1 = np.array(np.where(data.labels_==1)).flatten()
  idx_labels2 = np.array(np.where(data.labels_==2)).flatten()

  y[np.intersect1d(idx_train1, idx_labels0)] = 1 #defective type 1
  y[np.intersect1d(idx_train1, idx_labels2)] = 2 #defective type 2
  
  print(np.array(np.where(y==0)).flatten().shape)
  print(np.array(np.where(y==1)).flatten().shape)
  print(np.array(np.where(y==2)).flatten().shape)

In [ ]:
# use Keras function to split the arrays into training, test, and validation (70%, 15%, 15%)
y_cat = to_categorical(y)
desired_size = 500
frac = 1-desired_size/len(X)
#X_train, y_train = X,y_cat
X_train, X_valid_test, y_train, y_valid_test = train_test_split(X, y_cat, test_size = frac, random_state=1)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid_test, y_valid_test, test_size = 0.5, random_state=1)

print(X_train.shape, X_valid.shape, X_test.shape)
print(y_train.shape)
print(np.min(X_train[:,:,:,0]),np.max(X_train[:,:,:,0]))

In [ ]:
def augment_method(X, y, p):
  X_train_aug = []
  y_train_aug = []
  for i in range(len(X)):
    im = X[i]
    z = np.random.binomial(1,p)
    if z==1:
      x = np.random.uniform(0,1,1)
      if x<0.25: #shear
        val_x = float(np.random.uniform(0,0.3,1))
        val_y = float(np.random.uniform(0,0.3,1))
        transf = ((1, val_x, 0, val_y, 1, 0, 0, 0))
        im = tfa.image.transform(im, transf, fill_value = 150)
        #cv2_imshow(np.array(im))
      elif x>0.25 and x<0.5: #rotation
        angle = np.random.uniform(0,2*math.pi,1)
        im = tfa.image.rotate(im, angle, fill_value = 150)
        #cv2_imshow(np.array(im))
      elif x>0.5 and x<0.75: #crop and resize
        box = np.random.uniform(0.6,0.9,1)
        im = im[0:int(box*dim), 0:int(box*dim),:]
        im = tf.image.resize(im, (dim,dim))
        #cv2_imshow(np.array(im))
      else: #brightening
        delta = np.random.uniform(-0.7,0.7,1)
        im = tf.image.adjust_brightness(im, delta)
        #cv2_imshow(np.array(im))
      X_train_aug.append(im)
      y_train_aug.append(y[i])
  return np.array(X_train_aug), np.array(y_train_aug)

In [ ]:
def sample_mix(X, y, same_class, size, frac):
  if same_class:
    num_defective = np.count_nonzero(y == 1)
    num_ok = len(X) - num_defective
    tot = num_defective**2 + num_ok**2
    assert size < tot
    X_train_mix = []
    y_train_mix= []
    indices_defect = np.array(np.where(y==1)).flatten() #defective class mixing
    indices_normal = np.array(np.where(y==0)).flatten() #normal class mixing
    print(num_defective, num_ok)
    for i in range(num_defective):
      for j in range(num_defective):
        if i!=j and len(X_train_mix) < int(size*0.5):
            im = (X[indices_defect[i],:,:,:] + X[indices_defect[j],:,:,:])/2
            X_train_mix.append(im)
            y_train_mix.append(y[indices_defect[i]])
    for i in range(num_ok):
      for j in range(num_ok):
        if i!=j and len(X_train_mix) < size:
            im = (X[indices_normal[i],:,:,:] + X[indices_normal[j],:,:,:])/2
            X_train_mix.append(im)
            y_train_mix.append(y[indices_normal[i]])
  else:
    assert size < len(X)**2
    X_train_mix = []
    y_train_mix= []
    for i in range(len(X)):
      for j in range(len(X)):
        if i!=j and len(X_train_mix) < size:
            if frac:
              alpha = np.random.uniform(0.3,0.7,1)
            else:
              alpha = 0.5
            beta = 1-alpha
            im = (alpha*X[i,:,:,:] + beta*X[j,:,:,:])
            X_train_mix.append(im)
            if alpha>=0.5: #arbitrary choice here with =0.5
              y_train_mix.append(y[i]) 
            else:
              y_train_mix.append(y[j])
  return np.array(X_train_mix), np.array(y_train_mix)

In [ ]:
def random_del(X, y):
  X_train_del = []
  y_train_del= []
  height = X[0,:,:,:].shape[0]
  width = X[0,:,:,:].shape[1]
  for i in range(len(X)):
    coord1 = int(np.random.uniform(0,height,1))
    coord2 = int(np.random.uniform(0,width,1))
    len1 = int(np.random.uniform(coord1,height-coord1,1))
    len2 = int(np.random.uniform(coord2,width-coord2,1))
    im = X[i,:,:,:]
    im[coord1:coord1+len1,coord2:coord2+len2,:] = 0
    X_train_del.append(im)
    y_train_del.append(y[i])
  return np.array(X_train_del), np.array(y_train_del)

In [ ]:
def zca_whitening(X):
    #X_new = np.zeros((X.shape[0], X.shape[1], X.shape[2], X.shape[3]))
    X = X.reshape(X.shape[0], X.shape[1]*X.shape[2]*X.shape[3])
    for i in range(X.shape[0]):
      X_norm = X[i]/255
      X[i] = X_norm - X_norm.mean()
    # Covariance matrix [column-wise variables]: Sigma = (X-mu)' * (X-mu) / N
    sigma = np.cov(X, rowvar=False) # [M x M]
    print(sigma.shape)
    # Singular Value Decomposition. X = U * np.diag(S) * V
    U,S,V = scipy.linalg.svd(sigma, full_matrices=True)
        # U: [M x M] eigenvectors of sigma.
        # S: [M x 1] eigenvalues of sigma.
        # V: [M x M] transpose of U
    # Whitening constant: prevents division by zero
    epsilon = 0.1
    # ZCA Whitening matrix: U * Lambda * U'
    X_ZCA = U.dot(np.diag(1.0/np.sqrt(S + epsilon))).dot(U.T).dot(X_norm.T).T # [M x M]
    X_ZCA_rescaled = ((X_ZCA - X_ZCA.min()) / (X_ZCA.max() - X_ZCA.min()))
    X_ZCA_rescaled = X_ZCA_rescaled.reshape(X.shape[0], X.shape[1], X.shape[2], X.shape[3])
    #X_new[i,:,:,:] = X_ZCA_rescaled
    return X_ZCA_rescaled

In [ ]:
# ZCA = tf.keras.preprocessing.image.ImageDataGenerator(zca_whitening=True,zca_epsilon=1e-06)
# ZCA.fit(X_train)

In [ ]:
from contextlib import nullcontext
#Use k-means clustering
def run_kmeans(K, niter, image):
    
    # Load and transform an image
    img = image.astype('uint8')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width, nchannel = img.shape
    #scale = 500/max(height, width) # The longer side will be resized to 500
    #img = cv2.resize(img, (int(width*scale), int(height*scale)))

    # Vectorize the image
    x = img.reshape((-1, 3)).astype(np.float32)
    
    # Run K-means clustering
    labels, centroids = kmeans(x, K, niter)
    
    if labels is not None:
      # Produce the resulting image segmentation. 
      centroids = np.uint8(centroids)
      labels = np.int8(labels)
      result = centroids[labels.flatten()]
      result_image = result.reshape((img.shape))
      return result_image
    else:
      return None
    # Visualize the original image and the segmentation.
    # plt.figure(figsize=(5, 5))
    # plt.subplot(1,2,1)
    # plt.imshow(img); plt.title('Original Image'); plt.axis('off')
    # plt.subplot(1,2,2)
    # plt.imshow(result_image); plt.title('Segmented Image (K={})'.format(K)); plt.axis('off')
    # plt.show()

def kmeans(x, K, niter, seed=123):

    np.random.seed(seed)
    idx = np.random.choice(len(x), K, replace=False)

    # Randomly choose centroids
    centroids = x[idx, :]

    # Initialize labels
    labels = np.zeros((x.shape[0], ))
    changed = True
    i = 0
    
    while changed:
        
        if i == niter - 1:
            changed = False
            
        X_norms = np.array([np.sum(np.square(x),axis=1)])
        # Square of the matrix elements and summing across the rows
        centroids_norms = np.array([np.sum(np.square(centroids),axis=1)])
        # Finding xi*wj by doing X*WT
        X_dot_centroids = 2*np.matmul(x,np.transpose(centroids))

        distances = np.transpose(X_norms) + centroids_norms - X_dot_centroids
        temp = np.argmin(distances, axis = 1)
        
        if np.array_equal(temp, labels):
            changed = False 
        
        labels = temp
        centroid_labels, centroid_counts = np.unique(labels, return_counts=True)
        indx = np.isin(np.arange(K), centroid_labels)

        indx = indx.flatten()
        # number of times each centroid is associated with a point
        counts = np.zeros(K)
        # The index of the count matrix corresponds to the number of times the entry appears in u2
        #print(indx.shape, counts.shape, centroid_counts.shape)
        if (centroid_counts.shape!=counts.shape):
          return None, None
 
        counts[indx] = centroid_counts[indx]
        # adding 1e-20 to avoid division by 0 in final line
        counts = counts.reshape(K, 1) + 1e-20 
        # vectorized way of calculating sum over rows associated with each cluster
        sums = (np.eye(K)[labels]).T @ x
        centroids = sums/counts
        
        i+=1
    
    return labels, centroids

In [ ]:
sep = False

same_class = True
size = 2*len(X_train)
frac = False

mix = False
if mix:
  if sep:
    X_mix, y_mix = sample_mix(X_train,y_train, same_class, size, frac)
    print(X_train.shape, y_train.shape)
    print(X_mix.shape, y_mix.shape)
  else:
    X_mix, y_mix = sample_mix(X_train,y_train, same_class, size, frac)
    print(X_train.shape, y_train.shape)
    print(X_mix.shape, y_mix.shape)
    X_train = np.concatenate((X_train,X_mix))
    y_train = np.concatenate((y_train,y_mix))
    print(X_train.shape, y_train.shape)

augment = False
if augment:
  if sep:
    X_augment, y_augment = augment_method(X_train,y_train,1)
    print(X_train.shape, y_train.shape)
    print(X_augment.shape, y_augment.shape)
  else:
    X_augment, y_augment = augment_method(X_train,y_train,1)
    print(X_train.shape, y_train.shape)
    print(X_augment.shape, y_augment.shape)
    X_train = np.concatenate((X_train,X_augment))
    y_train = np.concatenate((y_train,y_augment))
    print(X_train.shape, y_train.shape)

rand_del = False
if rand_del:
  if sep:
    X_del, y_del = random_del(X_train,y_train)
    print(X_train.shape, y_train.shape)
    print(X_del.shape, y_del.shape)
  else:
    X_del, y_del = random_del(X_train,y_train)
    print(X_train.shape, y_train.shape)
    print(X_del.shape, y_del.shape)
    X_train = np.concatenate((X_train,X_del))
    y_train = np.concatenate((y_train,y_del))
    print(X_train.shape, y_train.shape)

ZCA = False
if ZCA:
  X_train = zca_whitening(X_train[0:10,:,:,:])

GAN = False
X_GAN = np.zeros_like(X_train)
y_GAN = np.zeros(X_train.shape[0])
if GAN:
  GAN_photos = '/content/drive/Shareddrives/Senior Thesis/GAN_Images/'
  i = 0
  for file in os.listdir(GAN_photos):
    while i < len(X_train):
      path = (GAN_photos+file)
      im = plt.imread(path)
      scaled_im = rescale(i, im, width, height)
      scaled_im = scaled_im[:,:,np.newaxis]
      scaled_im = np.repeat(scaled_im, 3, axis = 2)
      X_GAN[i] = scaled_im
      y_GAN[i] = 1
      i += 1
  if sep:
    print(X_train.shape, y_train.shape)
    print(X_GAN.shape, y_GAN.shape)
  else:
    print(X_train.shape, y_train.shape)
    print(X_GAN.shape, y_GAN.shape)
    X_train = np.concatenate((X_train,X_GAN))
    y_train = np.concatenate((y_train,y_GAN))
    print(X_train.shape, y_train.shape)

kmeans_do = False
if kmeans_do:
  X_kmeans = []
  y_kmeans = []
  for i in range(len(X_train)):
    #print(X_train[i,:,:,:].shape)
    #print(i)
    im = run_kmeans(2,10,X_train[i,:,:,:])
    #cv2_imshow(im)
    if im is not None:
      X_kmeans.append(im)
      y_kmeans.append(y[i])
  X_kmeans = np.array(X_kmeans)
  y_kmeans = np.array(y_kmeans)

from sklearn.decomposition import PCA
dims = 50
PCA_go = False
if PCA_go:
  X_train_fit = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]*X_train.shape[3])
  pca = PCA(n_components=dims)
  X_train_fit = pca.fit_transform(X_train_fit)
  #X_train_fit = X_train_fit.reshape(X_train.shape[0], X_train.shape[1],X_train.shape[2],X_train.shape[3])

  # X_valid_fit = X_valid.reshape(X_valid.shape[0], X_valid.shape[1]*X_valid.shape[2]*X_valid.shape[3])
  # pca = PCA(n_components=dims)
  # X_valid_fit = pca.fit_transform(X_valid_fit)
  #X_valid_fit = X_valid_fit.reshape(X_valid.shape[0], X_valid.shape[1],X_valid.shape[2],X_valid.shape[3])

In [ ]:
# print(X_kmeans.shape)
# desired_size = 100
# frac = 1-desired_size/len(X)
# X_train, y_train = X,y
# X_train, X_valid_test, y_train, y_valid_test = train_test_split(X_kmeans, y_kmeans, test_size = frac, random_state=1)
# X_valid, X_test, y_valid, y_test = train_test_split(X_valid_test, y_valid_test, test_size = 0.5, random_state=1)

# print(X_train.shape, X_valid.shape, X_test.shape)

In [ ]:
f, axarr = plt.subplots(2,2,figsize=(3,3))
for i in range(4):
  row = int(i/2)
  col = int(i%2)
  axarr[row,col].imshow(X[i+10,:,:,0], cmap='gray')
plt.show()

In [ ]:
# We build the base model
from keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
#X_train = tf.keras.applications.vgg16.preprocess_input(X_train)
#X_valid = tf.keras.applications.vgg16.preprocess_input(X_valid)

In [ ]:
import keras.backend as K
def custom_loss(y_true, y_pred):       

    loss = K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)
    FN = K.mean(K.greater(y_true, y_pred)) #Total False Negatives / Total Batch Size
    # FN = np.logical_and(y_true == 1, y_pred == 0)
    loss = loss + 3*FN
    return loss

In [ ]:
# We freeze every layer in our base model so that they do not train, we want that our feature extractor stays as before --> transfer learning
for layer in base_model.layers: 
  layer.trainable = False
  print('Layer ' + layer.name + ' frozen.')
# We take the last layer of our the model and add it to our classifier
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax', name='predictions')(x)
model = tf.keras.Model(base_model.input, x)

# We compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss_entropy = tf.keras.losses.CategoricalCrossentropy()
loss_binary_entropy = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss_entropy, metrics=['accuracy', 'Precision', 'Recall', 'AUC'])
#model.summary()
from keras.utils.vis_utils import plot_model
#plot_model(model, to_file='classifier_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# We start the training
#tf.config.experimental_run_functions_eagerly(True)
config = 1
transform = "forSegmentation"
preprocess = ""
imsize = str(dim)
path_name = f"{'/content/drive/Shareddrives/Senior Thesis/Models/my_model_'}{config}{'_'}{preprocess}{'_'}{transform}{'_'}{imsize}{'.hdf5'}"
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = tf.keras.callbacks.ModelCheckpoint(path_name, save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')
sep = False
if sep:
  for i in range(5):
    model.fit(x=X_mix, y=y_mix, batch_size=128, epochs=8, verbose=1, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_data=(X_valid,y_valid), shuffle=True)
    model.fit(x=X_train, y=y_train, batch_size=128, epochs=2, verbose=1, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_data=(X_valid,y_valid), shuffle=True)
else:
  model.fit(x=X_train, y=y_train, batch_size=128, epochs=50, verbose=1, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_data=(X_valid,y_valid), shuffle=True)

In [ ]:
print(path_name)

In [ ]:
plot = False
accuracy = model.history.history['accuracy']
val_accuracy = model.history.history['val_accuracy']
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']
precision = model.history.history['precision']
recall = model.history.history['recall']
if plot:
  epochs = range(len(accuracy))
  plt.plot(epochs, accuracy, 'b', label='Training accuracy')
  plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.ylim(0,1.0)
  plt.figure()
  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()

In [ ]:
def unfreeze_model(model):
    # We unfreeze the top 10 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-5:]:
        #if type(layer) == model.layers.BatchNormalization:
        layer.trainable = True

    #optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    #model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
unfreeze = False
if unfreeze:
  unfreeze_model(model)
  epochs = 20
  batch_size = 128

  # We now fine-tune the training
  mcp_save = tf.keras.callbacks.ModelCheckpoint(path_name, save_best_only=True, monitor='val_loss', mode='min')
  model.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_data=(X_valid,y_valid), shuffle=True)
  
  # plot training and validation losses and accuracy versus number of epochs
  accuracy_full = model.history.history['accuracy']
  val_accuracy_full = model.history.history['val_accuracy']
  loss_full = model.history.history['loss']
  val_loss_full = model.history.history['val_loss']
  precision_full = model.history.history['precision']
  recall_full = model.history.history['recall']


  accuracy_tot = np.concatenate((np.array(accuracy),np.array(accuracy_full)))
  val_accuracy_tot = np.concatenate((np.array(val_accuracy),np.array(val_accuracy_full)))
  loss_tot = np.concatenate((np.array(loss),np.array(loss_full)))
  val_loss_tot = np.concatenate((np.array(val_loss),np.array(val_loss_full)))
  precision_tot = np.concatenate((np.array(precision),np.array(precision_full)))
  recall_tot = np.concatenate((np.array(recall),np.array(recall_full)))

In [ ]:
#classifier = keras.models.load_model('/content/drive/Shareddrives/Senior Thesis/Models/my_model_2___100.hdf5')
# y_val=model.predict(X_valid)
# from sklearn.metrics import roc_curve,roc_auc_score, det_curve
# fpr_a , tpr , thresholds_a = roc_curve(y_valid , y_val, pos_label = 1)
# fpr_b , fnr , thresholds_b = det_curve(y_valid , y_val, pos_label = 1)

In [ ]:
import pandas as pd

if unfreeze:
  N = len(accuracy_tot)
  K = max(N, len(fpr_b))
  data = np.zeros((K,10))
  data[:N,0] = accuracy_tot
  data[:N,1] = val_accuracy_tot
  data[:N,2] = loss_tot
  data[:N,3] = val_loss_tot
  data[:N,4] = precision_tot
  data[:N,5] = recall_tot
  data[:len(fpr_a),6] = fpr_a
  data[:len(tpr),7] = tpr
  data[:len(thresholds_a),8] = thresholds_a
  data[:len(fpr_b),9] = fpr_b
  data[:len(fnr),10] = fnr
  data[:len(thresholds_b),11] = thresholds_b
else:
  N = len(accuracy)
  # K = max(N, len(fpr_b))
  data = np.zeros((N,12))
  data[:N,0] = accuracy
  data[:N,1] = val_accuracy
  data[:N,2] = loss
  data[:N,3] = val_loss
  data[:N,4] = precision
  data[:N,5] = recall
  # data[:len(fpr_a),6] = fpr_a
  # data[:len(tpr),7] = tpr
  # data[:len(thresholds_a),8] = thresholds_a
  # data[:len(fpr_b),9] = fpr_b
  # data[:len(fnr),10] = fnr
  # data[:len(thresholds_b),11] = thresholds_b

df = pd.DataFrame(data)
df.to_excel(excel_writer = "/content/data.xlsx")

In [ ]:
import time
start = time.time()
model.predict(X)
print((time.time()-start)/len(X))